In [ ]:
import torch
import numpy as np
from C3D import C3D
import time
import data_processing_sol as data_processing
import os
import os.path
from os.path import join
import pickle
from torch.utils.data import DataLoader
from torch.nn import functional as F

#Run the code using selected GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

TRAIN_CHECK_POINT = 'check_point/'

#Experiment, Optimization options
DATA_SPLIT_PATH = 'data_split.pkl'
BATCH_SIZE = 10
NUM_CLASSES = 11
CROP_SIZE = 112
CHANNEL_NUM = 3
CLIP_LENGTH = 16
EPOCH_NUM = 50
LEARNING_RATE = 1e-4

## 1) Load UCF11(UCF YouTube Action) Dataset Path

In [ ]:
DATA_SPLIT_PATH = 'data_split.pkl'
ucf11_dataset = pickle.load(open(DATA_SPLIT_PATH,'rb'))
train_set = ucf11_dataset['train']
test_set = ucf11_dataset['test']

## 2) Set Dataset and Dataloader

In [ ]:
train_video_dataset = data_processing.UCF11Dataset(train_set, NUM_CLASSES)
test_video_dataset = data_processing.UCF11Dataset(test_set, NUM_CLASSES)

train_video_dataloader = DataLoader(train_video_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_video_dataloader = DataLoader(test_video_dataset, batch_size = BATCH_SIZE, shuffle=True)

## 3) Set Network and Optimizer

In [ ]:
net = C3D(num_classes=NUM_CLASSES)
net = net.cuda()

optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

### - Train and Test C3D

In [ ]:
for epoch in range(EPOCH_NUM):
    # train
    correct_epoch = 0
    loss_epoch = 0
    net.train()
    
    for i, batch in enumerate(train_video_dataloader):        
        batch_clips = batch['clips']
        batch_labels = batch['labels']
        batch_clips = batch_clips.cuda()
        batch_labels = batch_labels.cuda()
        
        logits = net(batch_clips)                

        loss = F.cross_entropy(logits, batch_labels)
        correct = (torch.argmax(logits, 1) == batch_labels).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_epoch += loss
        correct_epoch += correct
        
        if i % 10 == 0:
            print('Epoch %d, Batch %d: Loss is %.5f; Accuracy is %.5f'%(epoch+1, i, loss, correct/batch_clips.shape[0]))
            
    print('Epoch %d: Average loss is: %.5f; Average accuracy is: %.5f'%(epoch+1, loss_epoch / len(train_video_dataloader),
                                                                                correct_epoch / len(train_video_dataset)))
                
    # test
    correct_epoch = 0
    loss_epoch = 0
    net.eval()
    
    with torch.no_grad():
        for i, batch in enumerate(test_video_dataloader):
            batch_clips = batch['clips']
            batch_labels = batch['labels']
            batch_clips = batch_clips.cuda()
            batch_labels = batch_labels.cuda()

            logits = net(batch_clips)

            loss = F.cross_entropy(logits, batch_labels)
            correct = (torch.argmax(logits, 1) == batch_labels).sum()    

            loss_epoch += loss
            correct_epoch += correct
        
    print('Test loss is %.5f; Accuracy is %.5f'%(loss_epoch / len(test_video_dataloader),
                                                                                correct_epoch / len(test_video_dataset)))


### 